<h1 style="text-align:center">   
      <font color = purple >📱 Tablet Fiyat Tahmini 📱 </font>    
</h1>
<hr style="width:100%;height:5px;border-width:0;color:gray;background-color:gray">
<center><img src="https://media.giphy.com/media/fOYqmZhNF2cGmWIiTK/giphy.gif"></center>

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import missingno  
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
from sklearn import ensemble
from sklearn.metrics import confusion_matrix as cm

filterwarnings('ignore')

from matplotlib.legend_handler import HandlerLine2D

In [ ]:
pip install missingno

Veri çerçevemizi bulunduğumuz dizinden yükleyelim ve bir veri çerçevesi haline getirerek df değişkenine atayalım. (pd.read_csv(...csv))

In [ ]:
tablet = pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df = tablet.copy()

Veri çerçevesinin ilk 5 gözlemini görüntüleyelim.

In [ ]:
df.head()

In [ ]:
df.shape
print('gozlem degeri :',df.shape[0],' degisken : ',df.shape[1])

Veri çerçevesindeki değişkenlerin hangi tipte olduğunu ve bellek kullanımını görüntüleyelim.

In [ ]:
df.info()


İnfo () 'yu kullandıktan sonra, bazı değişkenlerin tipinin object olduğunu görüyoruz. Bu değerleri makine öğrenmesinde sayısal değerlere dönüştürmemiz gerekir.

Veri çerçevesindeki sayısal değişken için temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

→ Sayısal verilerin standart sapmaları, ortalamaları, minumun, maximum değerlerini görüntülüyoruz. İncelersek; sırasıyla "OnKameraMP " , "ArkaKameraMP" , "CozunurlukYükseklik" gözlemlerinin minumun 0 olduğunu görebiliriz.Bu değerlerin gerçekten 0 olma ihtimali var mı ? Tabiki yok bu durum kamera olmaması olarak düşünülebilir. Kamer ayoksa 0 denmiş olabilir ya da değerler girilmemiş olabilir. Bu ihtimalleri değerlendirmek için benzersiz değerleri görüntüleyelim. Daha sonra fikir yürütebiliriz. Aynı zamanda standart sapmanın çok yüksek olduğunu görüyoruz.


→ "Kalinlik" değişkenine bakarsak; min 0.1 max 1.0 arasında değerlerini görüyoruz. Minumum değeri beni düşündürdü çünkü cm cinsinden yapılsa 0.1 cm olabilir mi emin değilim.

In [ ]:
df.isna().sum()

Benzersiz değerleri ve dağılımları görüntüleyelim.

In [ ]:
df["Bluetooth"].unique()

In [ ]:
df[(df["OnKameraMP"] == 0)]

→ "OnKameraMP" gözlemine baktığımızda; öncesinde 5 adet değerini olmadığını biliyoruz. "df[(df["OnKameraMP"] == 0)]" sonucunda 471 gözlemin 0 olduğunu görüyoruz. 471 tabletin ön kamerası bulunmuyor olabilir ya da değeri girilmemiş olabilir.

In [ ]:
df["ArkaKameraMP"].unique()

In [ ]:
df[(df["ArkaKameraMP"] == 0)]

→ "ArkaKameraMP" gözlemine baktığımızda; öncesinde eksik değerinin olmadığını biliyoruz. "df[(df["ArkaKameraMP"] == 0)]" sonucunda 101 gözlemin 0 olduğunu görüyoruz. 101 tableti arka kamera değeri 0 mp olara girilmiş.

In [ ]:
df["CozunurlukYükseklik"].unique() 

In [ ]:
df[(df["CozunurlukYükseklik"] == 0)]

In [ ]:
df["Bluetooth"].value_counts()

In [ ]:
df["CiftHat"].unique()

In [ ]:
df["CiftHat"].value_counts()

In [ ]:
df["4G"].unique()

In [ ]:
df["4G"].value_counts()

In [ ]:
df["3G"].unique()

In [ ]:
df["3G"].value_counts()

In [ ]:
df["Dokunmatik"].unique()

In [ ]:
df["Dokunmatik"].value_counts()

In [ ]:
df["WiFi"].unique()

In [ ]:
df["WiFi"].value_counts()

In [ ]:
df["FiyatAraligi"].unique()

In [ ]:
df["FiyatAraligi"].value_counts()

In [ ]:
df["Renk"].unique()

Tüm unique değerlere bakarsak dağılımlarının genelde orantılı olduğunu söyleyebiliriz.

In [ ]:
df["Renk"].value_counts()

Korelasyon matrisi çizdirelim.

In [ ]:
corr = df.corr()
corr

--> Korelasyon 1.0'e yaklaştıkça değerler arasındaki ilişki o kadar güçlenir. Burada Arka kamera ile ön kamera arasındaki değer 0.64569. Burada korelasyon ilişkisi en yüksek olan iki özniteliğimiz bunlar ama bu bile orta düzey bir ilişkidir. Bu nedenle özniteliklerimizin arasında iyi bir ilişki olduğunu söyleyemeyiz.

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

--> Isı haritasına bakarsak neredeyse tamamı siyah renkte yani öznitelikler arasında ilişki yok denecek kadar zayıf. Tek turuncu renk arka kamera ve ön kamera arasında.

Değişkenlerin veri türlerini hatırlayalım daha sonra sayısal değişkenelrin grafiklerini çizdirelim

In [ ]:
df.info()

Özniteliklerimizi kendi içinde sırasıyla değerlendirelim.

In [ ]:
df['FiyatAraligi'].value_counts()

In [ ]:
sns.countplot(x = "FiyatAraligi", data = df);

--> Hepsinden eşit sayıda gözlem var yani dengeli bir dağılım var.

In [ ]:
sns.distplot(df["ArkaKameraMP"], bins=16, color="purple");

-->0 ila 20 arasında değer alan arka kamera mp değerinin dengeli bir şekilde dağıldığını söyleyebiliriz. 


In [ ]:
sns.distplot(df["DahiliBellek"], bins=16, color="gold");

In [ ]:
sns.distplot(df["MikroislemciHizi"], bins=16, color="gold");

--> Grafiğe bakarsak; MikroislemciHizi değerlerinin 0.5 ile 3.0 GHz frekansları arasında değer aldığını görebiliriz. 0.5'e yakın kısımda bir yoğunluk olduğunu görüyoruz. Bunun dışında değerlerin dengeli dağıldığını görüyoruz. Önceki grafikte tüm fiyat aralıklarında tablet sayısının eşit olduğunu biliyorduk bu nedenle farklı fiyat segmentinde tabletlerin aynı işlemci hızına sahip olduğunu söyleyebiliriz.

In [ ]:
sns.distplot(df["OnKameraMP"], bins=16, color="blue");

--> OnKameraMP değerinde 471 adet 0 değerinin olduğunu biliyorduk bu nedenle 0'a yakın değerlerin daha yoğun olduğunu görüyoruz.

In [ ]:
sns.distplot(df["CozunurlukYükseklik"], bins=16, color="blue");

In [ ]:
sns.jointplot(x = "ArkaKameraMP", y = "OnKameraMP", data = df, color = "blue");

--> Aralarında güçlü pozitif ilişki bulunan ön ve arka kamera özniteliklerimize baktığımız zaman arka kamera mp değerlerinin her zaman ön kamera değerlerinden fazla olduğunu görüyoruz. Gerçek hayatta düşünürsek de bu durum geçerlidir. Ön kamera değerleri çok yüksek değerlere ulaşamıyorken arka kamera değerlerinin ulaştığını söyleyebiliriz.

Bir de bu iki özniteliğin yoğunluk haritasına bakalım.

In [ ]:
sns.jointplot(x = "ArkaKameraMP", y = "OnKameraMP", data = df, color = "blue", kind = "kde");

-->Yoğunluk haritasına bakarak aralarında pozitif bir etkileşim olsa da arka kamera mp değerinin ön kamera mp değerinden daha yüksek oranlarda arttığını söyleyebiliriz.

Kategorik değişkenlerimize bakalım.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "RAM", data = df);

--> Fiyatı daha yüksek olan cihazlarda RAM miktarı daha fazladır.Ram miktarının fiyata doğrudan bir etkisi var.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "BataryaGucu", data = df);

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "DahiliBellek", data = df); 

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "MikroislemciHizi", data = df);

--> Üç grafiğe baktığımızda değerlerde anormal diyebileceğimiz bir sonuç görmüyoruz yani diğer öznitelikler farklı fiyat aralıklarında benzer özelliklere sahip bu bize sınıflandırma yaparken bu özelliklerin aslında çok da belirleyici bir özelliği olmadığınını gösterir. Her kategoride aynı sonuçları vermiştir ama özellikle RAM özniteliğinde kategorilere göre RAM miktarlarının yoğunluğunun değiştiğini görüyoruz bu özniteliğin sınıflandırma yaparken bizim için daha değerli bir özellik olacağını öngörebiliriz.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "CekirdekSayisi", data = df);

Kullanılan teknolojilerin fiyat aralığını belirleyen en temel unsurlar olduğunu biliyoruz. Bir de onlara bakalım. Öznitelikleri hatırlayalım

In [ ]:
df.axes 

In [ ]:
sns.countplot(df["FiyatAraligi"]); 

In [ ]:
sns.countplot(df["Bluetooth"]);

In [ ]:
sns.countplot(df["4G"]);

In [ ]:
sns.countplot(df["Dokunmatik"]);

--> Fiyat araligi gibi **dengeli dağıldığını görüyoruz.

In [ ]:
sns.factorplot("FiyatAraligi", "CozunurlukYükseklik","WiFi", data = df, kind = "bar");

FiyatAraligi' nı önceden inceleyip dengeli olduğunu görmüştük. Bu bizim için çok iyi bir şey.

Fiyat Performans üzerine bir değerlendirme yapalım. Ram, batarya gücü ve fiyat arasındaki ilişkiyi scatterplot ile inceleyelim.

In [ ]:
sns.scatterplot(x = "RAM", y = "BataryaGucu",hue = "FiyatAraligi",data = df);

Belli ki fiyat arttıkça ram miktarı da artıyor. Biraz daha dikkatli baktığımızda ise ram miktarı normalin biraz üstünde olsa dahi batarya gücünün fazlalığı sebebiyle fiyatlandırmadaki pahalılığı görebiliyoruz. Buna dayanarak batarya gücü de fiyatı doğru orantılı olarak etkiliyor diyebiliriz.

Eksik verilere tekrar bakalım.

In [ ]:
df.isna().sum()

In [ ]:
!pip install missingno

In [ ]:
missingno.matrix(df,figsize=(20, 10));

In [ ]:
missingno.heatmap(df, figsize= (15,8));

--> Önceki incelememizde öznitelikler arasında kuvvetli ilişki olmadığını görmüştük.Burada 0'a yakın bir ilişki gözlemliyoruz.

Bir fonksiyon oluşturalım ve bu fonksiyonla boş olan değerler yüzde olarak hangi değerde olduğunu inceleyelim yüzde olarak büyük bir eksiği görürsek kullanışlı olmayacağını düşünürsek silelim. Eğer yüzde olarak küççükse doldurmayı tercih edebiliriz.

In [ ]:
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son

In [ ]:
eksik_deger_tablosu(df)

Yüzdelik olarak değerleri çok küçük hatta önemsiz denecek değerler olduğundan silmeyelim bu değerleri doldurmayı deneyelim. Ondan önce RAM özniteliğinin benzersiz değerlerine bir bakalım.

In [ ]:
df["RAM"].unique()

In [ ]:
df["RAM"].nunique()

In [ ]:
sns.barplot(x = "FiyatAraligi", y = "RAM", data = df);

--> Ramdeki eksik verileri doldurmamız gerektiğini buradan da anlıyoruz.

Öncelikle boş olan değerlerimizin diğer öznitelikler özellikle de FiyatAraligi ile nasıl bir ilişkisi olduğunu inceleyelim ve boş değeri buna göre dolduralım.

In [ ]:
sns.countplot(df[df["OnKameraMP"].isnull()]["FiyatAraligi"]);

In [ ]:
df.groupby("FiyatAraligi")[["OnKameraMP"]].mean()

--> Ön kamera özelliğinin boş değerlerinin hepsi "Çok ucuz kategorisinde". Buradan verimli çıkarım yapmak için çok az verimiz var 5 adet.Fakat bunlardan çıkarım yapacak olursak ;
Çok ucuz tablet kategorisinde ön kameranın 0 MP değerine sahip olmuş olabilir. Belki de bu yüzden bu veriler girilmemiştir. Tabii ki bu çıkarımın kesinliğinden emin olmadığımız için ve bu gözlemlerden yararlanmak için boş verileri "Çok ucuz tablet" kategorisindeki tabletlerin ortalama değerleriyle dolduracağız.

In [ ]:
sns.countplot(df[df["RAM"].isnull()]["FiyatAraligi"]);

In [ ]:
df.groupby("FiyatAraligi")[["RAM"]].mean()

--> Ram özelliğindeki boş veriler "Pahalı fiyat aralığı"ndaki tabletlerin değerlerine denk geldi. Ortalamayı değiştirmeden bu boş değerlere ortalama değerler ile doldurarak veri eksikliğini giderebiliriz.

In [ ]:
df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())]

In [ ]:
df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())]

In [ ]:
CokUcuz_OnKameraMP = df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["OnKameraMP"].isnull())].index

In [ ]:
Pahalı_RAM = df[(df["FiyatAraligi"] == "Pahalı") & (df["RAM"].isnull())].index

--> Şimdi sıra geldi boş değerleri kendi bulduğumuz değerler ile doldurmaya burada ".loc" fonksiyonunu kullanacağım. DataFrame nesneleri satır ve sütun adlarına sahip olduğu için Numpy’ dan farklı olarak bu satır ve sütun adlarıyla istediğimiz veriyi seçebiliriz. Seçme işlemleri için “.loc[]” kullanılabilir bu fonksiyon girilen kordinatları belirlenen değerler ile doldurmamızı sağlıyor. Biz de indexlerimizi ve öznitelikleri girerek o null değerleri dolduruyoruz.

In [ ]:
df.loc[CokUcuz_OnKameraMP ,"OnKameraMP"] = 4.1
df.loc[Pahalı_RAM, "RAM"] = 3449


--> Burada "Ön kamera" nın ortalama değeri 4.09'du fakat çok yakın bir değer olan 4.1'e yuvarlayıp verileri doldurduk. Ramde'de aynı şekilde 3449 ortalama değeriyle doldurduk. Şimdi eksik verileri görüntüleyelim.

In [ ]:
df.isna().sum()

--> Yukarıda da gördüğümüz üzere boş kalan bir verimiz yok.

--> Bu noktada çözünürlük verilerinin öznel olduğunu söylemeliyiz. Çözünürlük verilerin Özel ekran olan 35:9 monitör boyutu olduğunu düşünüyoruz ve buradan yola çıkarak bu oranın (0.25 ~) altındaki tüm verileri hatalı kabul edip bir standarta uyduracağız. Oran diye bir değişken oluşturarak devam edelim.

In [ ]:
oran = df['CozunurlukYükseklik'] / df['CozunurlukGenislik']

In [ ]:
df['ekranOrani'] = oran
df

In [ ]:
df[(df["ekranOrani"] < 0.25)]

In [ ]:
df['CozunurlukYükseklik'].std()

In [ ]:
df["ekranOrani"].mean()

In [ ]:
df.groupby("FiyatAraligi")[["CozunurlukYükseklik"]].mean()

--> Çözünürlük Yüksekliğinin ve Ekran Oranı gözemlerini ortalamasına bakalım çünkü ilerledikçe bu değerlerin nasıl değiştiğini gözlemlemek istiyoruz.

In [ ]:
df.groupby("FiyatAraligi")[["CozunurlukGenislik"]].mean()

In [ ]:
pahali_cozunurluk = df[(df["FiyatAraligi"] == "Pahalı") & (df["ekranOrani"] < 0.25)].index
ucuz_cozunurluk = df[(df["FiyatAraligi"] == "Ucuz") & (df["ekranOrani"] < 0.25)].index
normal_cozunurluk = df[(df["FiyatAraligi"] == "Normal") & (df["ekranOrani"] < 0.25)].index
cokUcuz_cozunurluk = df[(df["FiyatAraligi"] == "Çok Ucuz") & (df["ekranOrani"] < 0.25)].index

--> Tüm 0.25 değeri altında kalan gözlemlerin indislerini fiyatlarına göre değişkenlerde tuttuk.

In [ ]:
df.loc[pahali_cozunurluk ,"CozunurlukYükseklik"] = df['CozunurlukGenislik'] * 0.510424
df.loc[ucuz_cozunurluk ,"CozunurlukYükseklik"] = df['CozunurlukGenislik'] * 0.534434
df.loc[normal_cozunurluk ,"CozunurlukYükseklik"] = df['CozunurlukGenislik'] * 0.510424
df.loc[cokUcuz_cozunurluk ,"CozunurlukYükseklik"] = df['CozunurlukGenislik'] * 0.479679

--> Belirlediğimiz sınrın altında kalan tüm indisleri almıştık. Onların "CozunurlukYükseklik" özelliklerine daha önceden hesapladığımız ekran oranları ile çarpıp güncelliyoruz.

In [ ]:
df.head(10)

--> Gerçekte küsüratlı bir megapiksel değeri olamayacağı için çözünürlükleri "astype" ile integer değere çeviriyoruz.

In [ ]:
df["CozunurlukYükseklik"] = df["CozunurlukYükseklik"].astype(int)

--> Şu an Ekran oranı özniteliği eski çözünürlük değerlerine göre sonuç veriyor bu yüzden Ekran oranı özniteliğini güncelleyeceğim.

In [ ]:
oran2 = df['CozunurlukYükseklik'] / df['CozunurlukGenislik']
df['ekranOrani'] = oran2
df.sample(10)

--> Gözlemlere baktığımızda 0.25 in altında değer olmadığını görebiliriz. bunu "Ekran Oranı" ile sağlamıştık artık ekran oranı ile işimiz kalmadı onu bi kaç satır sonra drop (silebiliriz) edebiliriz.

--> Çözünürlük Yüksekliğinin STD'si 443tü bu değeri 371 e düşürdük. bu düzeltmeler ile veriyi daha sağlıklı hale getirdik.

In [ ]:
df["CozunurlukYükseklik"].std()

In [ ]:
df["ekranOrani"].mean()

In [ ]:
df.groupby("FiyatAraligi")[["ekranOrani"]].mean()

In [ ]:
df.groupby("FiyatAraligi")[["CozunurlukYükseklik"]].mean()

--> "CozunurlukYükseklik" in standart sapmasının azaldığını fakat diğer niteliklerinde artış yaşadığını görüyoruz. Bu normal bir durum çünkü 0.25'ten küçük değerleri yükselttik.

Ekran Oranını silelim.

In [ ]:
df.drop(["ekranOrani"], axis = 1, inplace = True)

In [ ]:
df.sample(10)

RAM ve Ön Kameradaki eksik verilerimizi doldurduk. Ekran Çözünürlüğü ile ilgili kısımda da tutarsız verileri bir ortalamalar ile düzenledik. ama verisetimizde bundan daha fazla eksik var mı boxplotlarla aykırı değerleri ( possible outliers ) ları inceleyelim

--> Box plot, aynı anda birden fazla değişkenin dağılımını görselleştirmek için harika bir yoldur, ancak genişlik / noktasallıktaki sapmayı bu görselleştirmeyi kullanarak belirlemek zordur.

In [ ]:
sns.boxplot(x = df['OnKameraMP']);

In [ ]:
sns.boxplot(x = df['CozunurlukYükseklik']);

<h1>Dummy Variable Oluşturma 

In [ ]:
df.columns

In [ ]:
df['Bluetooth'] = pd.Categorical(df['Bluetooth'])
dfDummies = pd.get_dummies(df['Bluetooth'], prefix = 'Bluetooth')
dfDummies
#pd.get_dummies(),veri işleme için kullanılır. Kategorik verileri kukla veya gösterge değişkenlere dönüştürür.
#prefix = 'Bluetooth' , sütun adlarına ön ek eklemek için kullanılır

--> Bluetooth özniteliğimizi kategorize ettiğimizde rahatlıkla 0 1 ler yardımıyla her gözlemde kolayca bluetooh'un var olup olmadığını inceleyebiliyoruz.

Gözlemlerimizin sonuna bu kolonları da concat edelim.

In [ ]:
df = pd.concat([df, dfDummies], axis=1)
df.head()

Modelde yer alan nitel vasıflı değişkenlere kukla değişken denir. Kukla değişkenler genel olarak ikil/binom (kadın,erkek) ve çoklu (mezuniyet: ilköğretim, ortaöğretim, lise, lisans, lisansüstü vb) biçimde iki yaklaşımla ölçülür.Binom ölçeklendirmede nitel değişken 1 ve 0 durumunu alır(kadın=1, erkek=0 gibi). Çoklu (polychotomous) ölçeklendirmede ise değişkene ait niteliksel vasıflar uygun ve aralarında anlamsal bir farklılığı yansıtacak bir biçimde sıralanır( ilköğretim =0, ortaöğretim=1, lise=2, lisans=3, lisansüstü=4 gibi).

Modelde eğer nitel bir değişkenin ikili yapıda tanımlarını kullanacak isek, model sağlıklı bir biçimde tahmin edilemez çünkü katsayılardan oluşan matrisinin tersi alınamaz. **Modelin sağlıklı bir biçimde tahmin edilebilmesi için genelde kontrol değişkeni olarak tanımlanan kukla değişkenini modelden düşürmemiz gerekir. Yani sizin araştırmadaki amacınız kadın olma vasfının bağımlı değişken üzerindeki etkisi is tabloda verilen "cinsiyet" ve "erkek" sütunlarını kullanmamamız gerekir.**

-->Tabloda da fazlalık oluşturmasına gerek olmadığından "bluetooth_yok"u silelim.Bluetooth'un var olduğu bilgisi bize yeter.

In [ ]:
df.drop(['Bluetooth', 'Bluetooth_Yok'], axis = 1, inplace = True)

--> Yukarıda bluetooh'a uyguladığımız işlemleri geri kalan özniteliklere de uygulayalım.

In [ ]:
df['3G'] = pd.Categorical(df['3G'])
dfDummies = pd.get_dummies(df['3G'], prefix = '3G')
df = pd.concat([df, dfDummies], axis=1)
df.drop(['3G', '3G_Yok'], axis = 1, inplace = True) 

In [ ]:
df['4G'] = pd.Categorical(df['4G'])
dfDummies = pd.get_dummies(df['4G'], prefix = '4G')
df = pd.concat([df, dfDummies], axis=1)
df.drop(['4G', '4G_Yok'], axis = 1, inplace = True)

In [ ]:
df['CiftHat'] = pd.Categorical(df['CiftHat'])
dfDummies = pd.get_dummies(df['CiftHat'], prefix = 'CiftHat')
df = pd.concat([df, dfDummies], axis=1)
df.drop(['CiftHat', 'CiftHat_Yok'], axis = 1, inplace = True)

In [ ]:
df['WiFi'] = pd.Categorical(df['WiFi'])
dfDummies = pd.get_dummies(df['WiFi'], prefix = 'WiFi')
df = pd.concat([df, dfDummies], axis=1)
df.drop(['WiFi', 'WiFi_Yok'], axis = 1, inplace = True)

In [ ]:
df['Dokunmatik'] = pd.Categorical(df['Dokunmatik'])
dfDummies = pd.get_dummies(df['Dokunmatik'], prefix = 'Dokunmatik')
df = pd.concat([df, dfDummies], axis=1)
df.drop(['Dokunmatik', 'Dokunmatik_Yok'], axis = 1, inplace = True)

In [ ]:
df

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

-->İlk oluşturduğumuz heatmap ile karşılaştırdığımızda ÇözünürlükYükseklik ve Çözünürlük Genişlik değerlerindeki ilişkiyi ç güçlendirmek için tutarsız ve boş verileri mantıklı veriler ile doldurduğumuzdan güçlendirdiğimizi söyleyebiliriz. Burada diğer heatmapmizden farklı olarak 3G ve 4G arasında çok güçlü olmasa da güçlü bi ilişki görebiliyoruz.

-->En son oluşturduğumuz haliyle tablonun korelasyon ısı haritasını alırsak öncekinden farklı olarak Çözünürlük değerleri arasındaki ilişkinin daha fazla olduğunu gözlemleriz bu noktada tutarsız verileri kendimiz çözünürlük genişlik verisi ile belli bir oran ile çarpıp yerleştirdiğimizden olduğunu söyleyebiliriz.

-->Tabloda çok öznitelik olduğu için Renk özniteliğini heatmapten sonra yapalım. Aynı işlemleri uygulayalım.

In [ ]:
df['Renk'] = pd.Categorical(df['Renk'])
dfDummies = pd.get_dummies(df['Renk'], prefix = 'Renk')
df = pd.concat([df, dfDummies], axis=1)

In [ ]:
df.drop(['Renk', 'Renk_Yeşil'], axis = 1, inplace = True)

In [ ]:
df.columns

<h1> Naive Bayes Algoritması 

In [ ]:
y = df['FiyatAraligi'] # hedef özniteliğimiz.
X = df.drop(['FiyatAraligi'], axis=1) # çıkarımda bulunacağımız özniteliklerimiz.


In [ ]:
y

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

--> Test size'ı %25 , Train(öğrenme) ise %75 olarak kullanıyorum burada öğrenme veri kümesini küçük tutarak sonuçların çok iyi olmasını engelliyorum. Random state belirtmek bu noktada önemli belirtilmezse, kodu her çalıştırdığımızda yeni bir rastgele değer oluşturulur.

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()

Modelimizi oluşturup eğitim verilerimiz ile modelimizi eğitelim.

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)

Modelimizin alabileceği parametreleri görüntüleyelim.

In [ ]:
nb_model 

Veri çerçevemizdeki test verilerini tahmin ettirip ve y_pred değişkenine atıyorum.

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

--> Model şu an %74.6 oranında doğru tahminde bulunuyor. Karmaşıklık matrisi ile hangi sınıfı ne kadar doğru tahmin etmiş bir bakalım.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

--> Peki burada daha iyi sonuç alabilir miyiz ***Feature Selection*** yöntemi kullanarak modele etkisi düşük olan öznitelikleri kaldırmayı deneyelim. 
***Feature Selection***, tahmine dayalı bir model geliştirirken girdi değişkenlerinin sayısını azaltma işlemidir. Hem modellemenin hesaplama maliyetini azaltmak hem de bazı durumlarda modelin performansını iyileştirmek için girdi değişkenlerinin sayısını azaltmak arzu edilir.



In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
len(X.columns)

In [ ]:
test = SelectKBest(k = 29)
test
# 29 öznitelikten oluştuğu için 29 parametresini eğitimde kullandık

In [ ]:
fit = test.fit(X, y)
fit

In [ ]:
for indis, skor in enumerate(fit.scores_):
    print(skor, " -> ", X.columns[indis])

--> Skorları incelediğimizde tüm özniteliklerin çok az skora sahip olduklarını sadece RAM özniteliğinin 4 haneli olduğunu hatta bazılarının 0'a yakın değerler aldığını görüyoruz. Mesela burada renk özniteliğinin her değeri 0'a yakın değerlerde. Bu üyeleri silip tekrardan oluşturabilirim.

In [ ]:
y = df['FiyatAraligi']
X = df[["BataryaGucu", "CozunurlukYükseklik", "CozunurlukGenislik", "RAM", "WiFi_Var", "Dokunmatik_Var", "4G_Var", "3G_Var", "CiftHat_Var", "Bluetooth_Var", "BataryaOmru", "ArkaKameraMP", "CekirdekSayisi", "Agirlik", "Kalinlik", "DahiliBellek", "OnKameraMP", "MikroislemciHizi"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

In [ ]:
nb = GaussianNB()
nb_model = nb.fit(X_train, y_train)
nb_model

In [ ]:
y_pred = nb_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)


--> accuracy_score 0.818 yani  Renk özniteliğini çıkardığımızda modelimizin %7.2 oranında daha iyi sonuçlar verdiğini gözlemledik.

Karmaşıklık matrisini kullanarak modelin hangi sınıfı ne kadar doğru tahmin etmiş inceleyelim.

In [ ]:
cross_val_score(nb_model, X_test, y_test, cv = 10)

In [ ]:
cross_val_score(nb_model, X_test, y_test, cv = 10).mean()

--> Modelimizi K Fold Cross Validation yaparak 10 parçaya böldük ve sırayla bir değeri test olarak seçip diğer değerleri eğitim olarak döngü halinde kullanıyoruz. Skor olarak ta 0.7979 değerini aldık.



* Cross-validation, makine öğrenmesi modelinin görmediği veriler üzerindeki performansını mümkün olduğunca objektif ve doğru bir şekilde değerlendirmek için kullanılan istatistiksel bir yeniden örnekleme(resampling) yöntemidir.

Diğer skorlarımızı da görüntüleyelim.

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

--> Kesinlik (Precision) ise Positive olarak tahminlediğimiz değerlerin gerçekten kaç adedinin Positive olduğunu göstermektedir. %81.7 oranını yakalamışız. Bu değer aynı zamanda modelimizin ne kadar istikrarlı olduğu hakkında da bize bilgi verir.

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

--> Duyarlılık (Recall) ise Positive olarak tahmin etmemiz gereken işlemlerin ne kadarını Positive olarak tahmin ettiğimizi gösteren bir metriktir.

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

--> F1 Score değeri bize Kesinlik (Precision) ve Duyarlılık (Recall) değerlerinin harmonik ortalamasını göstermektedir.

Son olarak karmaşıklık matrisini tekrardan yazdıralım ve yaptığımız değişikliğin modelimize nasıl bir etkisi olduğunu inceleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

--> Yaptığımız bu değişiklik sayesinde; 
* Normal fiyat kategorisinde +12 
* Pahalı fiyat kategorisinde +15
* Ucuz fiyat kategorisinde +7 
* Çok ucuz kategorisinde +2 daha iyi sonuç aldığımızı görüyoruz.

--> Burada ROC skoru alamıyoruz çünkü bulmayı hedeflediğimiz şey bir object olarak geçiyor sayısal bir değer değil.

<h1> Decision Tree Algoritması

In [ ]:
cart = DecisionTreeClassifier(random_state = 42)
cart_model = cart.fit(X_train, y_train)

In [ ]:
cart_model

Veri çerçevesindeki tüm gözlemleri tahmin edip y_pred değişkenine atayalım.

In [ ]:
y_pred = cart_model.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test, y_pred)

Doğruluk skoru: 0.82

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cart_grid = {"max_depth": range(1,20),
            "min_samples_split" : range(2,100)}

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

PrecisionScore, RecallScore ve F1Score değerleri neredeyse aynı. Model güzel...

In [ ]:
cart = DecisionTreeClassifier()
cart_cv = GridSearchCV(cart, cart_grid, cv = 3, n_jobs = -1, verbose = 2)
cart_cv_model = cart_cv.fit(X_train, y_train)

In [ ]:
print("En iyi parametreler : " + str(cart_cv_model.best_params_))
print("En iyi skor : " + str(cart_cv_model.best_score_))

In [ ]:
cart = DecisionTreeClassifier(max_depth = 9, min_samples_split = 23)
cart_tuned = cart.fit(X_train, y_train)

In [ ]:
y_pred = cart_tuned.predict(X_test)

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test, y_pred)

Karmaşıklık matrisini görüntüleyelim...

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(cart_tuned, X, y, cv = 10)

In [ ]:
cross_val_score(cart_tuned, X, y, cv = 10).mean()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
PrecisionScore = precision_score(y_test, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y_test, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y_test, y_pred, average = 'weighted')  
F1Score

PrecisionScore, RecallScore ve F1Score değerleri neredeyse aynı.

In [ ]:
ranking = cart_tuned.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = X.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="black", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

Karar ağacımıza göre bu yarışmayı kazanan özelliğimiz RAM oluyor.

Modelimizi cart modele göre eğitelim.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, y_train)

In [ ]:
cart_model

In [ ]:
!pip install skompiler
!pip install graphviz
!pip install pydotplus
!pip install astor

In [ ]:
print(cart_model.predict)

In [ ]:
df.columns

y_pred e atayalım.

In [ ]:
y_pred = cart_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

Doğruluk skoru 0.834...

In [ ]:
karmasiklik_matrisi = confusion_matrix(y_test, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(cart_model, X, y, cv = 10)

In [ ]:
cross_val_score(cart_model, X, y, cv = 10).mean()

cross_val_score ve accuracy_score değerleri birbirine yakın çıktı...

In [ ]:
print(classification_report(y_test, y_pred))

Modelimiz veriyi çok iyi kavramış durumda.

Fiyat aralığı "Normal" olduğunda değerlerimiz farklı olmuş precision 0.75, recall 0.80 ve f1 score 0.77 değerlerine sahip.. Fiyat aralığı "Pahalı" olduğunda precision 0.89, recall 0.85 ve f1 score 0.87 değerlerinde... Fiyat aralığı "Ucuz" olduğunda bütün hesaplar eşit çıkmış. Fiyat aralığı "Çok Ucuz" olduğunda precision 0.91, recall 0.89 ve f1 score 0.90 değerlerde....

In [ ]:
ranking = cart.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = X.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="black", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

Modelimizi yeniden eğitelim...

In [ ]:
knn = KNeighborsClassifier()
knn_model = knn.fit(X_train, y_train)

In [ ]:
knn_model

y_pred e atayalım...

In [ ]:
y_pred = knn_model.predict(X)

In [ ]:
accuracy_score(y, y_pred)

Doğruluk skoru 0.9415.

In [ ]:
karmasiklik_matrisi = confusion_matrix(y, y_pred)
print(karmasiklik_matrisi)

In [ ]:
cross_val_score(cart_tuned, X, y, cv = 10)

In [ ]:
cross_val_score(cart_tuned, X, y, cv = 10).mean()

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
PrecisionScore = precision_score(y, y_pred, average='weighted')
PrecisionScore

In [ ]:
RecallScore = recall_score(y, y_pred, average='weighted')
RecallScore

In [ ]:
F1Score = f1_score(y, y_pred, average = 'weighted')  
F1Score

Genel olarak baktığımızda bu değerlerimiz birbirine yakın çıkıyor.

Modelimiz gayet iyi görünüyor. Değerlerimiz tamamen birbirine çok yakın durumdalar.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,15)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(10)
knn_tuned = knn.fit(X_train, y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

Başarı skorumuz 0.922 çıkmış..

In [ ]:
score_list = []

for each in range(2,15):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,y_train)
    score_list.append(knn2.score(X_test, y_test))
    
plt.plot(range(2,15),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()

Doğruluk skoru , k değerleri değiştikçe değişmekte... Sabit tek bir noktası olmuş. Skorun en yüksek olduğu nokta k=10 noktası , en düşük olduğu nokta k=2 noktasıdır.

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10)

In [ ]:
cross_val_score(knn_tuned, X_test, y_test, cv = 10).mean()

In [ ]:
ranking = cart.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = X.columns

plt.figure(figsize = (16, 9))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="orange", align="center")
plt.xticks(range(len(features)), columns[features], rotation = 80)
plt.show()